In [13]:
!pip install autokeras


In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf

import autokeras as ak


## A Simple Example
The first step is to prepare your data. Here we use the [Titanic
dataset](https://www.kaggle.com/c/titanic) as an example.


In [15]:

TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)


The second step is to run the
[StructuredDataClassifier](/structured_data_classifier).
As a quick demo, we set epochs to 10.
You can also leave the epochs unspecified for an adaptive number of epochs.


In [4]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=3
)  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(
    # The path to the train.csv file.
    train_file_path,
    # The name of the label column.
    "survived",
    epochs=10,
)
# Predict with the best model.
predicted_y = clf.predict(test_file_path)
# Evaluate the best model with testing data.
print(clf.evaluate(test_file_path, "survived"))


Trial 3 Complete [00h 00m 09s]
val_accuracy: 0.8608695864677429

Best val_accuracy So Far: 0.8782608509063721
Total elapsed time: 00h 00m 20s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
20/20 [==============================] - 1s 6ms/step - loss: 0.7024 - accuracy: 0.5008
Epoch 2/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5930 - accuracy: 0.7209
Epoch 3/10
20/20 [==============================] - 0s 6ms/step - loss: 0.5301 - accuracy: 0.7783
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4872 - accuracy: 0.8198
Epoch 5/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4582 - accuracy: 0.8214
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4405 - accuracy: 0.8230
Epoch 7/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4295 - accuracy: 0.8246
Epoch 8/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4219 - accuracy: 0.8246
Epoch 9/10
20/20 [==================

## Data Format
The AutoKeras StructuredDataClassifier is quite flexible for the data format.

The example above shows how to use the CSV files directly. Besides CSV files,
it also supports numpy.ndarray, pandas.DataFrame or [tf.data.Dataset](
https://www.tensorflow.org/api_docs/python/tf/data/Dataset?version=stable). The
data should be two-dimensional with numerical or categorical values.

For the classification labels,
AutoKeras accepts both plain labels, i.e. strings or integers, and one-hot encoded
encoded labels, i.e. vectors of 0s and 1s.
The labels can be numpy.ndarray, pandas.DataFrame, or pandas.Series.

The following examples show how the data can be prepared with numpy.ndarray,
pandas.DataFrame, and tensorflow.data.Dataset.


In [5]:

# x_train as pandas.DataFrame, y_train as pandas.Series
x_train = pd.read_csv(train_file_path)
print(type(x_train))  # pandas.DataFrame
y_train = x_train.pop("survived")
print(type(y_train))  # pandas.Series

# You can also use pandas.DataFrame for y_train.
y_train = pd.DataFrame(y_train)
print(type(y_train))  # pandas.DataFrame

# You can also use numpy.ndarray for x_train and y_train.
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
print(type(x_train))  # numpy.ndarray
print(type(y_train))  # numpy.ndarray

# Preparing testing data.
x_test = pd.read_csv(test_file_path)
y_test = x_test.pop("survived")

# It tries 10 different models.
clf = ak.StructuredDataClassifier(overwrite=True, max_trials=3)
# Feed the structured data classifier with training data.
clf.fit(x_train, y_train, epochs=10)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))


Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.852173924446106

Best val_accuracy So Far: 0.852173924446106
Total elapsed time: 00h 00m 11s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
20/20 [==============================] - 1s 3ms/step - loss: 0.6413 - accuracy: 0.7193
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5635 - accuracy: 0.7847
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5120 - accuracy: 0.7815
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4757 - accuracy: 0.7863
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4511 - accuracy: 0.8070
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4349 - accuracy: 0.8182
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4243 - accuracy: 0.8182
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4166 - accuracy: 0.8214
Epoch 9/10
20/20 [====================

The following code shows how to convert numpy.ndarray to tf.data.Dataset.


In [6]:
train_set = tf.data.Dataset.from_tensor_slices((x_train.astype(np.unicode), y_train))
test_set = tf.data.Dataset.from_tensor_slices(
    (x_test.to_numpy().astype(np.unicode), y_test)
)

clf = ak.StructuredDataClassifier(overwrite=True, max_trials=3)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=10)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))


Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.8608695864677429

Best val_accuracy So Far: 0.886956512928009
Total elapsed time: 00h 00m 17s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
20/20 [==============================] - 1s 4ms/step - loss: 0.6717 - accuracy: 0.6364
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5932 - accuracy: 0.7656
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5323 - accuracy: 0.8006
Epoch 4/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4826 - accuracy: 0.8150
Epoch 5/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4482 - accuracy: 0.8214
Epoch 6/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4285 - accuracy: 0.8262
Epoch 7/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4177 - accuracy: 0.8278
Epoch 8/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4111 - accuracy: 0.8357
Epoch 9/10
20/20 [===================

You can also specify the column names and types for the data as follows.  The
`column_names` is optional if the training data already have the column names,
e.g.  pandas.DataFrame, CSV file.  Any column, whose type is not specified will
be inferred from the training data.


In [7]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    column_names=[
        "sex",
        "age",
        "n_siblings_spouses",
        "parch",
        "fare",
        "class",
        "deck",
        "embark_town",
        "alone",
    ],
    column_types={"sex": "categorical", "fare": "numerical"},
    max_trials=10,  # It tries 10 different models.
    overwrite=True,
)



## Validation Data
By default, AutoKeras use the last 20% of training data as validation data.  As
shown in the example below, you can use `validation_split` to specify the
percentage.


In [8]:
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)


Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.8795180916786194

Best val_accuracy So Far: 0.9036144614219666
Total elapsed time: 00h 00m 47s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
20/20 [==============================] - 1s 4ms/step - loss: 0.6990 - accuracy: 0.6603
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4785 - accuracy: 0.8070
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4117 - accuracy: 0.8389
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3784 - accuracy: 0.8373
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3581 - accuracy: 0.8533
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3429 - accuracy: 0.8517
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3303 - accuracy: 0.8596
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3194 - accuracy: 0.8676
Epoch 9/10
20/20 [=================

You can also use your own validation set
instead of splitting it from the training data with `validation_data`.


In [9]:
split = 500
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)


## Customized Search Space
For advanced users, you may customize your search space by using
[AutoModel](/auto_model/#automodel-class) instead of
[StructuredDataClassifier](/structured_data_classifier). You can configure the
[StructuredDataBlock](/block/#structureddatablock-class) for some high-level
configurations, e.g., `categorical_encoding` for whether to use the
[CategoricalToNumerical](/block/#categoricaltonumerical-class). You can also do
not specify these arguments, which would leave the different choices to be
tuned automatically. See the following example for detail.


In [10]:

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
)
clf.fit(x_train, y_train, epochs=10)


Trial 3 Complete [00h 00m 03s]
val_loss: 0.4827429950237274

Best val_loss So Far: 0.4827429950237274
Total elapsed time: 00h 00m 10s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
16/16 [==============================] - 1s 3ms/step - loss: 0.6446 - accuracy: 0.6660
Epoch 2/10
16/16 [==============================] - 0s 3ms/step - loss: 0.5671 - accuracy: 0.7720
Epoch 3/10
16/16 [==============================] - 0s 3ms/step - loss: 0.5210 - accuracy: 0.7880
Epoch 4/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4917 - accuracy: 0.8040
Epoch 5/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4717 - accuracy: 0.8100
Epoch 6/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4574 - accuracy: 0.8100
Epoch 7/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4471 - accuracy: 0.8140
Epoch 8/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4397 - accuracy: 0.8140
Epoch 9/10
16/16 [==========================

The usage of [AutoModel](/auto_model/#automodel-class) is similar to the
[functional API](https://www.tensorflow.org/guide/keras/functional) of Keras.
Basically, you are building a graph, whose edges are blocks and the nodes are
intermediate outputs of blocks.
To add an edge from `input_node` to `output_node` with
`output_node = ak.[some_block]([block_args])(input_node)`.

You can even also use more fine grained blocks to customize the search space even
further. See the following example.


In [11]:

input_node = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=1)
clf.predict(x_train)


Trial 1 Complete [00h 00m 02s]
val_loss: 0.6903156042098999

Best val_loss So Far: 0.6903156042098999
Total elapsed time: 00h 00m 02s
INFO:tensorflow:Oracle triggered exit
16/16 [==============================] - 1s 3ms/step - loss: 4.9019 - accuracy: 0.4100
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
16/16 [==============================] - 0s 2ms/step


array([[0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],

You can also export the best model found by AutoKeras as a Keras Model.


In [12]:
model = clf.export_model()
model.summary()
print(x_train.dtype)
# numpy array in object (mixed type) is not supported.
# convert it to unicode.
model.predict(x_train.astype(np.unicode))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 9)                0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 32)                320       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 re_lu_1 (ReLU)              (None, 32)                0         
                                                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing this will not modify any behaviour and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


array([[0.3967356 ],
       [0.9998797 ],
       [0.45046955],
       [0.99913514],
       [0.32378644],
       [0.89940965],
       [0.6215802 ],
       [0.9636234 ],
       [0.85482746],
       [0.55445683],
       [0.99467814],
       [0.6920141 ],
       [0.91536564],
       [0.7346407 ],
       [0.8742254 ],
       [0.31275266],
       [0.9799287 ],
       [0.9880694 ],
       [0.9951339 ],
       [0.30940974],
       [1.        ],
       [0.2890243 ],
       [0.39093745],
       [0.98064893],
       [0.9999994 ],
       [0.2829625 ],
       [0.11723304],
       [0.99986964],
       [0.9993678 ],
       [0.30963743],
       [0.8396283 ],
       [0.26662543],
       [0.9556875 ],
       [0.34688956],
       [0.9776256 ],
       [0.39895338],
       [0.78222215],
       [0.94256616],
       [0.9290303 ],
       [0.98437726],
       [0.9999482 ],
       [0.97729254],
       [0.9999235 ],
       [0.9909526 ],
       [0.7339063 ],
       [0.304232  ],
       [0.9947324 ],
       [0.385

## Reference
[StructuredDataClassifier](/structured_data_classifier),
[AutoModel](/auto_model/#automodel-class),
[StructuredDataBlock](/block/#structureddatablock-class),
[DenseBlock](/block/#denseblock-class),
[StructuredDataInput](/node/#structureddatainput-class),
[ClassificationHead](/block/#classificationhead-class),
[CategoricalToNumerical](/block/#categoricaltonumerical-class).
